**Softmax Activation Function**  
For our model we want it to be a classifier, so we need a activaion function meant for classification. One of these is the Softmax activation function. Why another activation function? In this case the rectified linear unit is unbounded, not normalized with other units and exclusive. Not norlmalized means a number or value can be anything, an output of [23, 87, 220] is without context and exclusive means each output is independet of each other.  
  
To adress this lack of context, the softmax activation on output data can take non-normalized (uncalibrated) inputs and produce a normalized distribution of probabilities for our classes. Thedistribution returned by the softwax activation function represents a confidence score for each class and will add upp to 1.

The softmax function: S(i,j) = e^z(i,j) / sum e^z(i,l)

In [1]:
layer_outputs = [4.8, 1.21, 2.385]

First step is to exponentiate the outputs. We do this with Eulers constant *e* to the power of the given parameter: y = e^x

In [5]:
from numpy import math

# values from previous steps, when describing what a neural network is
layer_outputs = [4.8, 1.21, 2.385]

# e -mathematical constans
math.e

# For each value in a vector, calculate the the exponential value
exp_values = []
for output in layer_outputs:
    exp_values.append(math.e ** output)
print('exponentiated values:')
print(exp_values)

exponentiated values:
[121.51041751873483, 3.353484652549023, 10.859062664920513]


Exponentiation serves multiple purposes. For calculating probabilities, we need non-negative values. Take for example output [4.8, 1.21, -2.385], even after normalization, the last value will still be negative wince we just divide all of them by their sum. A negative probability (or confidence) does not make sense to us. An exponential value of any number is always non-negative - it returns 0 for negative infinity, 1 for the input of 0, and increases for positive values.

In [ ]:
# Next normalize. Jatka tästä!